In [2]:
%%html

<svg id="svg1"/>

<style>

  body {
    font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
    position: relative;
    background-color: #555;
  }

</style>
    
    <script src="http://d3js.org/d3.v3.min.js"></script>
    
    <script>

      // define sizes counts opacities and colors
      var width         = 960,
          height        = 500,
          cellsize      = 150,                  // average cell size
          cellpadding   = 15,                   // minimum distance between nodes
          bleed         = 250,                  // how far outside element to generate nodes
          fillopacity   = 0.50,
          strokeopacity = 0.50,
          cellsX        = Math.ceil((width+bleed*2)/cellsize),
          cellsY        = Math.ceil((height+bleed*2)/cellsize),
          x_gradient    = ['#333', '#87afaf', '#af5f5f', '#565656'],
          y_gradient    = ['#87afd7', '#af5f5f', '#333'];

      // generate random data set to fill window and a bit beyond
      var nodes = d3.range(cellsX*cellsY).map(function(d) {
                     var col = d % cellsX;
                     var row = Math.floor(d / cellsX);
                     var x   = Math.round(-bleed + col*cellsize + Math.random() * (cellsize - cellpadding*2) + cellpadding);
                     var y   = Math.round(-bleed + row*cellsize + Math.random() * (cellsize - cellpadding*2) + cellpadding);
                     return [x, y];
                   });

      // color iterpolation / generation based on cell centroid
      function color(x, y) {
        var color_x = d3.scale.linear()
                        .range(x_gradient)
                        .domain(d3.range(0, width, width/x_gradient.length));   //[-bleed, width+bleed]
        var color_y = d3.scale.linear()
                        .range(y_gradient)
                        .domain(d3.range(0, height, height/y_gradient.length)); //[-bleed, width+bleed]
        return d3.interpolateRgb(color_x(x), color_y(y))(0.5);
      }

      // generate and place svg container
      var svg = d3.select('#svg1')
                  .attr('id', 'voronoi')
                  .attr('width', width)
                  .attr('height', height)
                  .on("mousemove", function() { nodes[0] = d3.mouse(this); draw(); });

      // define triangle voronoi
      var voronoi = d3.geom.voronoi()
                      .clipExtent([[cellpadding, cellpadding], [width - cellpadding, height - cellpadding]])
                      .triangles(nodes);

      // define paths
      var path = svg.append("g").selectAll("path");

      // initial rendering
      draw();

      // draw the triangles
      function draw() {
        path = path.data(d3.geom.voronoi().triangles(nodes).map(function(d) { return "M" + d.join("L") + "Z"; }), String);

        path.enter().append("path")
            .style('fill', function(d, i) {
              var c = d.replace(/M/g,'').replace(/Z/g,'').replace(/L/g,',');
              var c = c.split(',');
              var x = (parseFloat(c[0]) + parseFloat(c[2]) + parseFloat(c[4]))/3;
              var y = (parseFloat(c[1]) + parseFloat(c[3]) + parseFloat(c[5]))/3;
              return color(x, y);
            })
            . style('stroke', function(d,i) {
              var c = d.replace(/M/g,'').replace(/Z/g,'').replace(/L/g,',');
              var c = c.split(',');
              var x = (parseFloat(c[0]) + parseFloat(c[2]) + parseFloat(c[4]))/3;
              var y = (parseFloat(c[1]) + parseFloat(c[3]) + parseFloat(c[5]))/3;
              return color(x, y);
            })
            .style('fill-opacity', function(d,i) {
              if (fillopacity != 1)
                return fillopacity;
            })
            .style('stroke-opacity', function(d,i) {
              if (strokeopacity != 1)
                return strokeopacity;
            })
            .attr("d", String);

        path.exit().remove();
      }

    </script>
